In [59]:
from telethon.tl.types import Message, User
import re

# Stuff for Telegram messages

In [3]:
from telethon import TelegramClient

# Use your own values here
api_id = 100000
api_hash = 'api_hash'

client = TelegramClient('messages_hist', api_id, api_hash)

assert client.connect()

if not client.is_user_authorized():
    phone_number = '+555555555'
    client.send_code_request(phone_number)
    myself = client.sign_in(phone_number, input('Enter code: '))

In [8]:
my_dialogs = client.get_dialogs()

load messages from telegram dialogs

In [151]:
messages = []
for dialog in my_dialogs:
    if(type(dialog.entity) == User):
        messages.append(client.get_message_history(dialog.entity, limit=None))
        print('added messages with {}'.format(dialog.entity.username))

/anaconda3/lib/python3.6/site-packages/telethon/telegram_client.py:1117: UserWarning: get_message_history is deprecated, use get_messages instead
  'get_message_history is deprecated, use get_messages instead'


added messages with yanasavva
added messages with al_veg
added messages with polinkatomm
added messages with None
added messages with govorushkaBot
added messages with pskliff
added messages with alexkondr
added messages with taXXeN22
added messages with None
added messages with BotFather
added messages with kseniapavlina
added messages with egolyshev
added messages with Xetd71
added messages with gazzon
added messages with Usstasikus
added messages with nataliazs
added messages with Dmitryi_Grishin
added messages with Oogonek
added messages with DvachRaveBot
added messages with vietng7
added messages with mmn_13
added messages with DataFest5RegistrationBot
added messages with None
added messages with Rauanna
added messages with blacKitten13
added messages with savvaanton
added messages with Desasterrus
added messages with fox0117
added messages with suchANiceDay
added messages with ivvan16
added messages with oscorbinka
added messages with serizanglii
added messages with None
added me

# save and load methods for messages data (including ids, time and entity stuff)

In [2]:
import pickle

In [154]:
def save_data(data, file_name='tg_messages.dat'):
    with open(file_name,'wb') as f:
        pickle.dump(data, f)

In [155]:
save_data(messages)

In [3]:
def load_data(file_name='tg_messages.dat'):
    with open(file_name, 'rb') as f:
        messages = pickle.load(f)
    return messages

In [38]:
loaded_messages = load_data()

In [39]:
print(loaded_messages[5][9])

Message(id=177887, to_id=PeerUser(user_id=359134637), date=datetime.utcfromtimestamp(1525665717), message='Лол', out=False, mentioned=False, media_unread=False, silent=False, post=False, from_id=36594062, fwd_from=MessageFwdHeader(date=datetime.utcfromtimestamp(1525665462), from_id=36594062, channel_id=None, channel_post=None, post_author=None, saved_from_peer=None, saved_from_msg_id=None), via_bot_id=None, reply_to_msg_id=None, media=None, reply_markup=None, entities=[], views=None, edit_date=None, post_author=None, grouped_id=None)


In [178]:
my_id = messages[0][0].from_id

In [40]:
def save_messages(data, file_name='tg_messages.txt'):
    with open(file_name, 'w') as f:
        for dialog in data:
#             dialog.reverse()
            iter_dialog = iter(dialog[::-1])
            current_id = dialog[0].from_id
            quest_mess = ""
            break_cond = False
            while not break_cond:
                try:
                    message = next(iter_dialog)
                    while message.from_id == current_id:
                        print('collecting question')
                        if message.message:
                            # print(message.message)
                            quest_mess += clean_message(message.message).strip() + ' '
                        try:
                            message = next(iter_dialog)
                        except:
                            print('stop iteration on question: {}'.format(quest_mess))
                            break_cond = True
                    else:
                        if quest_mess:
                            f.write(quest_mess + '\n')
                            print('question: {}'.format(quest_mess))
                        answ_mess = ""
                        while message.from_id != current_id:
                            print('collecting answer')
                            if message.message:
                                answ_mess += clean_message(message.message).strip() + ' '
                            try:
                                message = next(iter_dialog)
                            except:
                                if answ_mess:
                                    f.write(answ_mess + '\n')
                                    print('answer: {}'.format(answ_mess))
                                break_cond = True
                        if answ_mess:
                            f.write(answ_mess + '\n')
                            print('answer: {}'.format(answ_mess))
                        if message.message:
                            quest_mess = clean_message(message.message).strip() + ' '
                except:
                    break
            print('\n=====================\n')


In [69]:
def clean_message(message):
    # Remove new lines within message
    cleanedMessage = message.replace('\n',' ').lower()
    # Remove punctuation
    cleanedMessage = re.sub('([.,!?])','', cleanedMessage)
    # Remove multiple spaces in message
    cleanedMessage = re.sub(' +',' ', cleanedMessage)
    # Remove html tag
    cleanedMessage = re.sub('<br>',' ', cleanedMessage)
    return cleanedMessage

# stuff for vk messages

In [5]:
vk_messages = load_data('vk_messages.dat')

In [55]:
vk_messages = [messages for messages in vk_messages[::-1] if type(messages)== dict and messages.]

In [67]:
def save_vk_messages(data, file_name='vk_messages.txt', my_id=83784811):
    length = len(data)
    with open(file_name, 'w') as f:
        my_message, other_message, current_speaker = "","",0
        for index, message in enumerate(data):
            if message['from_id'] == my_id:
                if not my_message:
                    start_index = index - 1
                my_message += message['body'] + ' '
            elif my_message:
                for cnt in range(start_index, 0, -1):
                    tmp_mess = data[cnt]
                    if not tmp_mess['body']:
                        my_message, other_message, current_speaker = "","",0
                        break
                    if not current_speaker:
                        current_speaker = tmp_mess['from_id']
                    elif current_speaker != tmp_mess['from_id']:
                        other_message = clean_message(other_message)
                        my_message = clean_message(my_message)
                        f.write(other_message + '\n')
                        f.write(my_message + '\n')
                        break
                    other_message = ' ' + tmp_mess['body'] + ' ' + other_message
                my_message, other_message, current_speaker = "","",0
            if index % 100 == 0:
                print('saved {} messages, {} left'.format(index, length - index))
                    

In [70]:
save_vk_messages(vk_messages)

saved 0 messages, 241827 left
saved 100 messages, 241727 left
saved 200 messages, 241627 left
saved 300 messages, 241527 left
saved 400 messages, 241427 left
saved 500 messages, 241327 left
saved 600 messages, 241227 left
saved 700 messages, 241127 left
saved 800 messages, 241027 left
saved 900 messages, 240927 left
saved 1000 messages, 240827 left
saved 1100 messages, 240727 left
saved 1200 messages, 240627 left
saved 1300 messages, 240527 left
saved 1400 messages, 240427 left
saved 1500 messages, 240327 left
saved 1600 messages, 240227 left
saved 1700 messages, 240127 left
saved 1800 messages, 240027 left
saved 1900 messages, 239927 left
saved 2000 messages, 239827 left
saved 2100 messages, 239727 left
saved 2200 messages, 239627 left
saved 2300 messages, 239527 left
saved 2400 messages, 239427 left
saved 2500 messages, 239327 left
saved 2600 messages, 239227 left
saved 2700 messages, 239127 left
saved 2800 messages, 239027 left
saved 2900 messages, 238927 left
saved 3000 messages, 2

saved 49500 messages, 192327 left
saved 49600 messages, 192227 left
saved 49700 messages, 192127 left
saved 49800 messages, 192027 left
saved 49900 messages, 191927 left
saved 50000 messages, 191827 left
saved 50100 messages, 191727 left
saved 50200 messages, 191627 left
saved 50300 messages, 191527 left
saved 50400 messages, 191427 left
saved 50500 messages, 191327 left
saved 50600 messages, 191227 left
saved 50700 messages, 191127 left
saved 50800 messages, 191027 left
saved 50900 messages, 190927 left
saved 51000 messages, 190827 left
saved 51100 messages, 190727 left
saved 51200 messages, 190627 left
saved 51300 messages, 190527 left
saved 51400 messages, 190427 left
saved 51500 messages, 190327 left
saved 51600 messages, 190227 left
saved 51700 messages, 190127 left
saved 51800 messages, 190027 left
saved 51900 messages, 189927 left
saved 52000 messages, 189827 left
saved 52100 messages, 189727 left
saved 52200 messages, 189627 left
saved 52300 messages, 189527 left
saved 52400 me

saved 115700 messages, 126127 left
saved 115800 messages, 126027 left
saved 115900 messages, 125927 left
saved 116000 messages, 125827 left
saved 116100 messages, 125727 left
saved 116200 messages, 125627 left
saved 116300 messages, 125527 left
saved 116400 messages, 125427 left
saved 116500 messages, 125327 left
saved 116600 messages, 125227 left
saved 116700 messages, 125127 left
saved 116800 messages, 125027 left
saved 116900 messages, 124927 left
saved 117000 messages, 124827 left
saved 117100 messages, 124727 left
saved 117200 messages, 124627 left
saved 117300 messages, 124527 left
saved 117400 messages, 124427 left
saved 117500 messages, 124327 left
saved 117600 messages, 124227 left
saved 117700 messages, 124127 left
saved 117800 messages, 124027 left
saved 117900 messages, 123927 left
saved 118000 messages, 123827 left
saved 118100 messages, 123727 left
saved 118200 messages, 123627 left
saved 118300 messages, 123527 left
saved 118400 messages, 123427 left
saved 118500 message

saved 161300 messages, 80527 left
saved 161400 messages, 80427 left
saved 161500 messages, 80327 left
saved 161600 messages, 80227 left
saved 161700 messages, 80127 left
saved 161800 messages, 80027 left
saved 161900 messages, 79927 left
saved 162000 messages, 79827 left
saved 162100 messages, 79727 left
saved 162200 messages, 79627 left
saved 162300 messages, 79527 left
saved 162400 messages, 79427 left
saved 162500 messages, 79327 left
saved 162600 messages, 79227 left
saved 162700 messages, 79127 left
saved 162800 messages, 79027 left
saved 162900 messages, 78927 left
saved 163000 messages, 78827 left
saved 163100 messages, 78727 left
saved 163200 messages, 78627 left
saved 163300 messages, 78527 left
saved 163400 messages, 78427 left
saved 163500 messages, 78327 left
saved 163600 messages, 78227 left
saved 163700 messages, 78127 left
saved 163800 messages, 78027 left
saved 163900 messages, 77927 left
saved 164000 messages, 77827 left
saved 164100 messages, 77727 left
saved 164200 m

saved 232300 messages, 9527 left
saved 232400 messages, 9427 left
saved 232500 messages, 9327 left
saved 232600 messages, 9227 left
saved 232700 messages, 9127 left
saved 232800 messages, 9027 left
saved 232900 messages, 8927 left
saved 233000 messages, 8827 left
saved 233100 messages, 8727 left
saved 233200 messages, 8627 left
saved 233300 messages, 8527 left
saved 233400 messages, 8427 left
saved 233500 messages, 8327 left
saved 233600 messages, 8227 left
saved 233700 messages, 8127 left
saved 233800 messages, 8027 left
saved 233900 messages, 7927 left
saved 234000 messages, 7827 left
saved 234100 messages, 7727 left
saved 234200 messages, 7627 left
saved 234300 messages, 7527 left
saved 234400 messages, 7427 left
saved 234500 messages, 7327 left
saved 234600 messages, 7227 left
saved 234700 messages, 7127 left
saved 234800 messages, 7027 left
saved 234900 messages, 6927 left
saved 235000 messages, 6827 left
saved 235100 messages, 6727 left
saved 235200 messages, 6627 left
saved 2353